In [1]:
# Input library yang digunakan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Load Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

dir = '/content/drive/MyDrive/NLP-SMS/dataset/dataset_sms_spam_v1.csv'

Mounted at /content/drive


In [3]:
df = pd.read_csv(dir)
df.head()

,teks,label
0,[PROMO] Beli paket Flash mulai 1GB di MY TELKO...,2
1,2.5 GB/30 hari hanya Rp 35 Ribu Spesial buat A...,2
2,"2016-07-08 11:47:11.Plg Yth, sisa kuota Flash ...",2
3,"2016-08-07 11:29:47.Plg Yth, sisa kuota Flash ...",2
4,4.5GB/30 hari hanya Rp 55 Ribu Spesial buat an...,2


Preprocessing Text

In [4]:
# Case Folding
import re

def casefolding(text):
  text = text.lower() # Mengubah text jadi lowercase
  text = re.sub(r'https?://\S+|www\.\S+', '', text) # Menghapus URL
  text = re.sub(r'[-+]?[0-9]+', '', text) # Menghapus angka
  text = re.sub(r'[^\w\s]', '', text) # Menghapus karakter tanda baca
  text = text.strip()
  return text

In [5]:
# Lihat perbedaan before dan after case folding
raw_sample = df['teks'].iloc[252]
case_folding = casefolding(raw_sample)

print('Raw data\t : ', raw_sample)
print('Case Folding\t : ', case_folding)

Raw data	 :  Ass, Sy Randy. Mengenai mobil yg sdh sy liat kondisi'y , kebetulan kami berminat. Mohon hub Bapa sy utk dibicarakan hrg netnya; Dr.H.DARMAWAN 0812xxxxxxx
Case Folding	 :  ass sy randy mengenai mobil yg sdh sy liat kondisiy  kebetulan kami berminat mohon hub bapa sy utk dibicarakan hrg netnya drhdarmawan xxxxxxx


Normalization Text

In [6]:
key_norm = pd.read_csv('/content/drive/MyDrive/NLP-SMS/dataset/key_norm.csv')
# Mengubah kata yang disingkat menjadi kata aslinya
def text_normalize(text):
  text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0]
                   if (key_norm['singkat'] == word).any()
                   else word for word in text.split()
                   ])
  text = str.lower(text)
  return text

In [7]:
# Lihat perbedaan before dan after normalization
raw_data = df['teks'].iloc[252]
normalisasi = text_normalize(case_folding)

print('Raw data\t : ', raw_data)
print('Normalisasi\t : ', normalisasi)

Raw data	 :  Ass, Sy Randy. Mengenai mobil yg sdh sy liat kondisi'y , kebetulan kami berminat. Mohon hub Bapa sy utk dibicarakan hrg netnya; Dr.H.DARMAWAN 0812xxxxxxx
Normalisasi	 :  ass saya randy mengenai mobil yang sudah saya lihat kondisiy kebetulan kami berminat mohon hubungi bapa saya untuk dibicarakan harga netnya drhdarmawan xxxxxxx


Filtering (Stopword Removal)

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stop_factory = stopwords.words('indonesian')
more_stopword = ['tsel', 'gb', 'rb', 'btw'] # Menambahkan kata dalam stopword

def remove_stop_words(text):
    sw = stop_factory + more_stopword
    clean_words = []
    text = text.split()
    for word in text:
        if word not in sw:
            clean_words.append(word)
    return " ".join(clean_words)

In [9]:
# Lihat perbedaan before dan after stopword
raw_sample = df['teks'].iloc[696]
case_folding = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)

print('Raw data\t : ', raw_sample)
print('Case Folding\t : ', case_folding)
print('Filtering\t : ', stopword_removal)

Raw data	 :  Btw magicomnya yg sedang Gais, gaada yg gede
Case Folding	 :  btw magicomnya yg sedang gais gaada yg gede
Filtering	 :  magicomnya yg gais gaada yg gede


Stemming

In [10]:
!pip -q install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.8 MB/s eta 0:00:00


In [11]:
# Mengubah kata menjadi kata dasar
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
  text = stemmer.stem(text)
  return text

In [12]:
# Lihat perbedaan before dan after stemming
raw_sample = df['teks'].iloc[10]
case_folding = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)
stemming_text = stemming(stopword_removal)

print('Raw data\t : ', raw_sample)
print('Case Folding\t : ', case_folding)
print('Filtering\t : ', stopword_removal)
print('Stemming\t : ', stemming_text)

Raw data	 :  Anda akan berhenti berlangganan Paket Flash. Ketik FLASH<spasi>YA jika setuju. Tunggu SMS konfirmasi penonaktifan Paket Anda
Case Folding	 :  anda akan berhenti berlangganan paket flash ketik flashspasiya jika setuju tunggu sms konfirmasi penonaktifan paket anda
Filtering	 :  berhenti berlangganan paket flash ketik flashspasiya setuju tunggu sms konfirmasi penonaktifan paket
Stemming	 :  henti langgan paket flash ketik flashspasiya tuju tunggu sms konfirmasi nonaktif paket


Text Preprocessing Pipeline

In [13]:
def data_process(text):
    text = casefolding(text)
    text = text_normalize(text)
    text = remove_stop_words(text)
    text = stemming(text)
    return text

In [14]:
%%time
df['clean_teks'] = df['teks'].apply(data_process)

CPU times: user 6min 20s, sys: 1.21 s, total: 6min 21s
Wall time: 6min 48s


In [15]:
df

,teks,label,clean_teks
0,[PROMO] Beli paket Flash mulai 1GB di MY TELKO...,2,promo beli paket flash my telkomsel app extra ...
1,2.5 GB/30 hari hanya Rp 35 Ribu Spesial buat A...,2,rupiah ribu spesial pilih aktif promo sd novem...
2,"2016-07-08 11:47:11.Plg Yth, sisa kuota Flash ...",2,pulang hormat sisa kuota flash kb download myt...
3,"2016-08-07 11:29:47.Plg Yth, sisa kuota Flash ...",2,pulang hormat sisa kuota flash kb download myt...
4,4.5GB/30 hari hanya Rp 55 Ribu Spesial buat an...,2,rupiah ribu spesial pilih aktif buru skb
...,...,...,...
1138,"Yooo sama2, oke nanti aku umumin di grup kelas",0,yooo oke umumin grup kelas
1139,😁 sebelumnya ga ad nulis kerudung. Kirain warn...,0,nulis kerudung kirain warna jins
1140,Mba mau kirim 300 ya,0,mbak kirim ya
1141,nama1 beaok bwrangkat pagi...mau cas atay tra...,0,nama beaok bwrangkat pagimau cas atay tranfer


In [16]:
# Simpan data yang sudah dipreprocessing ke dalam file csv
df.to_csv('/content/drive/MyDrive/NLP-SMS/dataset/clean_data.csv', encoding='utf-8')

Feature Engineering

In [17]:
# Pisahkan kolom feature dan target
X = df['clean_teks']
y = df['label']

In [18]:
X

0       promo beli paket flash my telkomsel app extra ...
1       rupiah ribu spesial pilih aktif promo sd novem...
2       pulang hormat sisa kuota flash kb download myt...
3       pulang hormat sisa kuota flash kb download myt...
4                rupiah ribu spesial pilih aktif buru skb
                              ...                        
1138                           yooo oke umumin grup kelas
1139                     nulis kerudung kirain warna jins
1140                                        mbak kirim ya
1141        nama beaok bwrangkat pagimau cas atay tranfer
1142                                       nomor bri nama
Name: clean_teks, Length: 1143, dtype: object

In [19]:
y

0       2
1       2
2       2
3       2
4       2
       ..
1138    0
1139    0
1140    0
1141    0
1142    0
Name: label, Length: 1143, dtype: int64

Feature Extraction (TF-IDF dan N-Gram)

In [20]:
# Save model
import pickle

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Unigram
vec_TF_IDF = TfidfVectorizer(ngram_range=(1,1))
vec_TF_IDF.fit(X)

x_tf_idf = vec_TF_IDF.transform(X)

pickle.dump(vec_TF_IDF.vocabulary_,open("/content/drive/MyDrive/NLP-SMS/dataset/feature_tf-idf.sav", "wb"))

In [21]:
# Menampilkan vocabulary dari TF-IDF
vec_TF_IDF.vocabulary_

{'promo': 2295,
 'beli': 323,
 'paket': 2088,
 'flash': 870,
 'my': 1880,
 'telkomsel': 2875,
 'app': 162,
 'extra': 841,
 'kuota': 1549,
 'lte': 1652,
 'telpon': 2878,
 'mnthr': 1831,
 'buru': 480,
 'cek': 521,
 'tselmemytsel': 3013,
 'sk': 2691,
 'rupiah': 2503,
 'ribu': 2452,
 'spesial': 2750,
 'pilih': 2175,
 'aktif': 66,
 'sd': 2557,
 'november': 1989,
 'pulang': 2330,
 'hormat': 1116,
 'sisa': 2684,
 'kb': 1410,
 'download': 752,
 'mytelkomsel': 1882,
 'apps': 167,
 'kuotabeli': 1550,
 'hubung': 1140,
 'skb': 2692,
 'ekstra': 804,
 'pulsa': 2332,
 'internet': 1220,
 'bulan': 466,
 'sjk': 2690,
 'augsept': 217,
 'detail': 665,
 'iring': 1242,
 'tarif': 2841,
 'panjang': 2101,
 'hits': 1105,
 'armada': 180,
 'curi': 600,
 'hati': 1069,
 'tekan': 2869,
 'okcall': 2041,
 'informasi': 1191,
 'eks': 801,
 'loh': 1639,
 'internetan': 1221,
 'pakai': 2086,
 'volume': 3129,
 'ultima': 3066,
 'mbhr': 1738,
 'harga': 1057,
 'tariflokasi': 2843,
 'tselmefl': 3011,
 'coboy': 568,
 'jr': 1329,

In [22]:
# Lihat jumlah Feature
print(len(vec_TF_IDF.get_feature_names_out()))

3253


In [23]:
x1 = vec_TF_IDF.transform(X).toarray()
data_tabular_tf_idf = pd.DataFrame(x1, columns=vec_TF_IDF.get_feature_names_out())
data_tabular_tf_idf

,aa,aamiiiin,aamiin,ab,abadi,abai,abbee,abdul,acara,acaratks,...,yudisium,yuk,yuks,yuni,yunit,zalora,zarkasi,zjt,zona,ztkm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Feature Selection

In [24]:
x_train = np.array(data_tabular_tf_idf)
y_train = np.array(y)

In [25]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

chi2_features = SelectKBest(chi2, k=3000)
x_kbest_features = chi2_features.fit_transform(x_train, y_train)

# Untuk reduced features
print('Original Feature Number', x_train.shape[1])
print('Reduced feature Number', x_kbest_features.shape[1])

Original Feature Number 3253
Reduced feature Number 3000


In [26]:
# Menampilkan features beserta nilainya
feature = vec_TF_IDF.get_feature_names_out()
feature

Data = pd.DataFrame(chi2_features.scores_, columns=['Nilai'])
Data['Fitur'] = feature
Data

,Nilai,Fitur
0,0.835608,aa
1,0.419698,aamiiiin
2,1.558607,aamiin
3,0.716455,ab
4,0.800674,abadi
...,...,...
3248,1.180239,zalora
3249,0.503162,zarkasi
3250,0.716455,zjt
3251,2.917381,zona


In [27]:
# Mengurutkan nilai feature terbaik
Data.sort_values(by='Nilai', ascending=False)

,Nilai,Fitur
2088,49.062970,paket
1030,46.324101,hadiah
1549,45.593560,kuota
2178,39.438431,pin
1486,34.448410,klik
...,...,...
1520,0.044910,kopi
307,0.044468,bca
1694,0.031579,maksimal
531,0.013783,cepat


In [28]:
# Menampilkan feature yang terpilih berdasarkan nilai mask atau nilai tertinggi yang sudah ditetapkan pada chi square
mask = chi2_features.get_support()

new_feature = []

for bool, f in zip(mask, feature):
  if bool :
    new_feature.append(f)
  selected_feature = new_feature
selected_feature

['aa',
 'aamiiiin',
 'aamiin',
 'ab',
 'abadi',
 'abai',
 'abbee',
 'abdul',
 'acaratks',
 'account',
 'ada',
 'adapromo',
 'adi',
 'adik',
 'adison',
 'admin',
 'administrasi',
 'adminlte',
 'ado',
 'adrian',
 'adu',
 'aduh',
 'advertising',
 'aea',
 'aesthetic',
 'afbe',
 'affc',
 'afr',
 'afrika',
 'agam',
 'agen',
 'agendain',
 'agenpulsa',
 'ags',
 'agst',
 'agsts',
 'agt',
 'agtskinfodlj',
 'agua',
 'agun',
 'agus',
 'agust',
 'agustuskunjungi',
 'ahaha',
 'ahub',
 'aidzin',
 'aigoo',
 'air',
 'aja',
 'ajaa',
 'ajaaa',
 'ajabri',
 'ajak',
 'ajeng',
 'akang',
 'akangteteh',
 'akbar',
 'akreditasi',
 'akses',
 'aksi',
 'aktif',
 'aktifasi',
 'aktivasi',
 'aktivitas',
 'akucintaislam',
 'akumulasi',
 'akun',
 'akurasi',
 'akurat',
 'alaikum',
 'alaikumsaya',
 'alaiqum',
 'alam',
 'alamat',
 'alamsyah',
 'alesannya',
 'algoritma',
 'alhamdulillah',
 'alhuda',
 'ali',
 'aliando',
 'all',
 'allah',
 'allahaamiin',
 'alphard',
 'alquran',
 'alur',
 'aman',
 'amanda',
 'ambil',
 'amin',


In [29]:
# Buat vocabulary baru berdasarkan fitur yang terseleksi untuk mengenerate fitur vektor tfid pd proses deploy
new_selected_features = {}

for (k,v) in vec_TF_IDF.vocabulary_.items():
  if k in selected_feature:
    new_selected_features[k] = v
new_selected_features

{'promo': 2295,
 'beli': 323,
 'paket': 2088,
 'flash': 870,
 'my': 1880,
 'telkomsel': 2875,
 'app': 162,
 'extra': 841,
 'kuota': 1549,
 'lte': 1652,
 'telpon': 2878,
 'mnthr': 1831,
 'buru': 480,
 'cek': 521,
 'tselmemytsel': 3013,
 'sk': 2691,
 'rupiah': 2503,
 'ribu': 2452,
 'spesial': 2750,
 'pilih': 2175,
 'aktif': 66,
 'sd': 2557,
 'november': 1989,
 'pulang': 2330,
 'hormat': 1116,
 'sisa': 2684,
 'kb': 1410,
 'download': 752,
 'mytelkomsel': 1882,
 'apps': 167,
 'kuotabeli': 1550,
 'hubung': 1140,
 'skb': 2692,
 'ekstra': 804,
 'pulsa': 2332,
 'internet': 1220,
 'bulan': 466,
 'sjk': 2690,
 'augsept': 217,
 'detail': 665,
 'iring': 1242,
 'tarif': 2841,
 'panjang': 2101,
 'hits': 1105,
 'armada': 180,
 'curi': 600,
 'hati': 1069,
 'tekan': 2869,
 'okcall': 2041,
 'informasi': 1191,
 'eks': 801,
 'loh': 1639,
 'internetan': 1221,
 'pakai': 2086,
 'volume': 3129,
 'ultima': 3066,
 'mbhr': 1738,
 'harga': 1057,
 'tariflokasi': 2843,
 'tselmefl': 3011,
 'coboy': 568,
 'jr': 1329,

In [30]:
pickle.dump(new_selected_features, open("/content/drive/MyDrive/NLP-SMS/dataset/new_selected_features_tf-idf.sav", "wb"))

In [31]:
# Menampilkan fitur yang sudah diseleksi
df_selected_features = pd.DataFrame(x_kbest_features, columns=selected_feature)
df_selected_features

,aa,aamiiiin,aamiin,ab,abadi,abai,abbee,abdul,acaratks,account,...,yudisium,yuk,yuks,yuni,yunit,zalora,zarkasi,zjt,zona,ztkm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Modeling

In [32]:
selected_x = x_kbest_features
selected_x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [34]:
x = selected_x
y = df.label

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [35]:
# Menampilkan jumlah data traning dan data testing
print("Jumlah X_train\t: ", len(x_train))
print("Jumlah X_test\t: ", len(x_test))
print("Jumlah X_train\t: ", len(x_train))
print("Jumlah X_test\t: ", len(x_test))

Jumlah X_train	:  914
Jumlah X_test	:  229
Jumlah X_train	:  914
Jumlah X_test	:  229


In [36]:
# Proses training dengan Naive Bayes
text_algorithms = MultinomialNB()

model = text_algorithms.fit(x_train, y_train)

# Buat model prediksi
data_input = ("tolong belikan mama pulsa nomor as mama teman mama celaka kluarganya hubung mama ganti uangnyapenting")
data_input = data_process(data_input)

# Load
tfidf = TfidfVectorizer
loaded_vec = TfidfVectorizer(decode_error="replace", vocabulary=set(pickle.load(open("/content/drive/MyDrive/NLP-SMS/dataset/new_selected_features_tf-idf.sav", "rb"))))

hasil = model.predict(loaded_vec.fit_transform([data_input]))

if (hasil == 0):
  s = "SMS Normal"
elif (hasil == 1):
  s = "SMS Penipuan"
else:
  s = "SMS Promo"

print("Hasil Prediksi : \n", s)

Hasil Prediksi : 
 SMS Penipuan


Evaluasi Model

In [37]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

predicted = model.predict(x_test)

CM = confusion_matrix(y_test, predicted)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95       126
           1       0.89      0.89      0.89        66
           2       0.84      0.86      0.85        37

    accuracy                           0.92       229
   macro avg       0.90      0.90      0.90       229
weighted avg       0.92      0.92      0.92       229



In [38]:
# Simpan Model
pickle.dump(model,open("/content/drive/MyDrive/NLP-SMS/dataset/model_fraud.sav", "wb"))